### 查看GPU

In [1]:
!nvidia-smi

Thu May  5 13:53:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### 安裝需要的BERT套件

In [2]:
!pip install transformers -U
!pip install nlp -U
!pip install torch -U

     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 880 kB 35.9 MB/s 
     |████████████████████████████████| 6.6 MB 35.4 MB/s 
     |████████████████████████████████| 77 kB 6.9 MB/s 
     |████████████████████████████████| 596 kB 38.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=f8440218f881ec69a9f82ddc5edf8968ff5a88a8037b78f73be43fca88cb27c0
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.7 MB 5.4 MB/s 
     |████████████████████████████████| 212 kB 45.1 MB/s 


### 載入套件

In [3]:
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
from nlp import load_dataset, Dataset
import nlp
import torch
import random
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

### 讀取訓練資料集

In [4]:
df = pd.read_csv('train.csv')
dataset = Dataset.from_pandas(df)


In [5]:
df.sample(10)

,ID,Name,Description,label
344,344,欣利康_DLPC_調節代謝_升級版_軟膠囊,「啟動酒精分解酵素-國際期刊研究…DLPC連續7年動物研究(狒狒)…12隻狒狒+酒7年→10...,1
198,198,MIDOKAWA防彈可可健康飲,「實驗者於一周內減少3~5...。到了48公斤，原本63...。（主持人口述）：...一天一...,1
52,52,液體螺旋藻,「提升人體自然防護力，幫助抵禦病毒」,1
373,373,亞積美之立-100%野生馬基莓粉（贈鼠尾草籽）-『馬基莓花青素第一品牌』（頂級智利酒果）,「模糊…乾澀酸澀…藍光…天然紫外線…眼鏡族…高含量多酚…可保護植物免於紫外線、細菌、病毒的侵害」,1
471,471,紅妍飲超值組、比菲德士菌EX,「每瓶6，000mg水嫩膠原…拒絕鬆垮由內撐起蘋果肌…每包150億個活性乳酸菌…腹愁者救星…...,1
627,627,苦瓜複方膠囊食品?,天然草本複合配方，每1顆含有:1)苦瓜萃取物250毫克2)肉桂125毫克3)鉻100微克,0
207,207,『禮盒綜合款』極品銀耳燕窩飲膠原蛋白/珍珠粉【兩種口味各三瓶】贈送精美提袋,「多?類物質-表皮生長因子，簡生EGF被譽為『美容基因』...美肌保濕...很受減重族群的青...,1
897,897,https://tw.buy.yahoo.com/gdsale/GNC健安喜-DHA魚油60...,每顆含有600毫克的DHA。-5倍純化技術，不含重金屬汞、鎘、鉛和多氯聯苯。-本配方為容易吞...,0
59,59,venus輕纖錠、固捷敏UC-II,「(venus輕纖錠)…『改善腸胃道健康效應之評估』研究，針對70名受試者，包括54名便秘型...,1
788,788,輔酵素Q10-NK,輔酵素Q10(CoQ10)分布於人體全身，但隨著年齡增加，體內合成輔酵素Q10的濃度會逐漸減...,0


### 載入原始模型

In [6]:
model = BertForSequenceClassification.from_pretrained('hfl/chinese-bert-wwm-ext')

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/393M [00:00<?, ?B/s]

Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

###  載入分詞模型

In [7]:
tokenizer = BertTokenizerFast.from_pretrained('hfl/chinese-bert-wwm-ext')

Downloading:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

### 設定執行參數

In [8]:
RANDOM_SEED = 5
MAX_LEN = 512
EPOCHS=5
BATCH_SIZE = 8

### 把資料集分成訓練、測試、驗證

In [9]:
shuffled_ds = dataset.shuffle(RANDOM_SEED)
split_ds = shuffled_ds.train_test_split(test_size=0.2)
train_dataset = split_ds['train']
test_val_dataset = split_ds['test']
split_tv = test_val_dataset.train_test_split(test_size=0.5)
test_dataset = split_tv['train']
val_dataset = split_tv['test']

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

### 取一筆出來瞧瞧(已經打亂)

In [10]:
train_dataset[2]

{'Description': '「打擊脂肪…恢復輕盈…燃燒力…體重跟體脂都有在持續緩慢的下降中…體重比上個月少了快2公斤…享瘦…降低腹部脂肪…減肥…降低脂肪…難瘦的腹部脂肪…纖體…頑固的腹部脂肪…瘦腹部脂肪…瘦不下來…變瘦…脂肪分解能力…脂肪燃燒能力…棕色脂肪細胞…燃燒力…打擊頑固脂肪…腹部脂肪面積減少…脂肪DOWN…腰圍DOWN…體重DOWN…抑制脂肪合成…沒有燃燒完的能量會透過肝臟合成成為中性脂肪，而中性脂肪會轉換成體脂肪蓄積…抑制中性脂肪合成的效果…促進脂肪分解…燃燒…協助脂肪分解…體脂肪進入容易燃燒的狀態…促進脂肪燃燒…透過棕色脂肪細胞燃燒…50倍燃燒力…消除身體水腫…減肥效果…皮下脂肪↓…內臟脂肪↓…腹部脂肪面積減少…減少的內臟脂肪體積…腹部內臟脂肪面積…體重卻下降了…腰圍有變鬆…種卻減了一公斤…真的變鬆了…服用…纖體」',
 'ID': 288,
 'Name': 'DMJ輕纖葛花錠',
 'label': 1}

In [36]:
tokenizer.tokenize("體重跟體脂都有在持續緩慢的下降中")

['體',
 '重',
 '跟',
 '體',
 '脂',
 '都',
 '有',
 '在',
 '持',
 '續',
 '緩',
 '慢',
 '的',
 '下',
 '降',
 '中']

### 定義分詞器

In [11]:
def tokenize(batch):
  return tokenizer(batch['Description'], max_length=MAX_LEN, padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

### 定義評測準則，訓練器

In [12]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=50,
    weight_decay=0.01,
    # evaluate_during_training=True,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

### 開始訓練

In [13]:
trainer.train()
trainer.evaluate()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 749
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 470


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 94
  Batch size = 8


{'epoch': 5.0,
 'eval_accuracy': 0.9787234042553191,
 'eval_f1': 0.9824561403508771,
 'eval_loss': 0.10244306921958923,
 'eval_precision': 1.0,
 'eval_recall': 0.9655172413793104,
 'eval_runtime': 5.7956,
 'eval_samples_per_second': 16.219,
 'eval_steps_per_second': 2.071}

### 將模型存檔

In [14]:
model.save_pretrained("bert_food_ad")

Configuration saved in bert_food_ad/config.json
Model weights saved in bert_food_ad/pytorch_model.bin


### 用TENSORBOARD查看訓練結果

In [ ]:
# %reload_ext tensorboard
%tensorboard --logdir='./logs' # --host 192.168.1.113
#from tensorboard import notebook
#notebook.list()
#notebook.display(port=6006, height=1000)

### 載入已訓練好的模型

In [15]:
from transformers import BertConfig

test_config = BertConfig.from_json_file('./bert_food_ad/config.json')
test_model = BertForSequenceClassification.from_pretrained('./bert_food_ad/pytorch_model.bin', config=test_config)
model.eval()

loading weights file ./bert_food_ad/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at ./bert_food_ad/pytorch_model.bin.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [16]:

print(df.iloc[220]['label'], df.iloc[220]['Description'])
print(df.iloc[578]['label'], df.iloc[578]['Description'])

1 「增強免疫力...護眼明目...抗氧化...保肝...皮膚：...保護皮膚，減緩皮屑問題，降低敏感...毛髮……促進毛髮亮麗...消化系統」「強化免疫系統，有助調整過敏體質...清除自由基延緩老化...調整腸胃道機能...調理皮膚問題...諾麗果含有許多強大的抗氧化劑」「阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外」「它能維護身體細胞組織正常運作，阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外，使身體恢復正常」
0 ·???????每份含有30mg輔Q10。輔Q10可參與維生素E再生，兩者具協同作用，可促進新陳代謝。保護循環順暢維持健康。暢銷日本的最佳美容聖品。


### 在此輸入測試文字

In [32]:
test_text = '增強免疫力...護眼明目...抗氧化...保肝...皮膚：...保護皮膚，減緩皮屑問題，降低敏感...毛髮……促進毛髮亮麗...消化系統」「強化免疫系統，有助調整過敏體質...清除自由基延緩老化...調整腸胃道機能...調理皮膚問題...諾麗果含有許多強大的抗氧化劑」「阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外」「它能維護身體細胞組織正常運作，阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外，使身體恢復正常'

MAX_LEN = 512

encoded_review = tokenizer.encode_plus(
    test_text,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    return_attention_mask=True,
    return_tensors='pt',
    truncation=True
)

In [33]:
encoded_review

{'input_ids': tensor([[ 101, 1872, 2485, 1048, 4554, 1213,  119,  119,  119, 6362, 4706, 3209,
         4680,  119,  119,  119, 2834, 3709, 1265,  119,  119,  119,  924, 5498,
          119,  119,  119, 4649, 5604, 8038,  119,  119,  119,  924, 6362, 4649,
         5604, 8024, 3938, 5227, 4649, 2244, 1558, 7539, 8024, 7360,  856, 3130,
         2697,  119,  119,  119, 3688, 7773,  100,  100,  914, 6868, 3688, 7773,
          778, 7927,  119,  119,  119, 3867, 1265, 5143, 5186,  520,  519, 2485,
         1265, 1048, 4554, 5143, 5186, 8024, 3300, 1221, 6310, 3146, 6882, 3130,
         7768, 6549,  119,  119,  119, 3926, 7370, 5632, 4507, 1825, 2454, 5227,
         5439, 1265,  119,  119,  119, 6310, 3146, 5591, 5517, 6887, 3582, 5543,
          119,  119,  119, 6310, 4415, 4649, 5604, 1558, 7539,  119,  119,  119,
         6330, 7927, 3362, 1419, 3300, 6258, 1914, 2485, 1920, 4638, 2834, 3709,
         1265, 1212,  520,  519, 7349, 3632, 2772, 3938, 2208, 4567, 4578, 4638,
         4634,

### 進行測試

In [31]:
test_model = test_model.to('cuda')

input_ids = encoded_review['input_ids'].to('cuda')
attention_mask = encoded_review['attention_mask'].to('cuda')
output = test_model(input_ids, attention_mask)
result = torch.argmax(output[0][0])

classnames = ['Passed', 'Failed']

print(f'Review text: {test_text}')
print(f'Result  : {classnames[result]}')

Review text: 讓你完全恢復正常
Result  : Passed


In [ ]:
!nvidia-smi

Tue May 11 09:09:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    62W / 149W |   5234MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------